goal: save hf models from remote to local disk, so that I can populate the directory with evaluation results.

In [2]:
import platform
import sys
from huggingface_hub import snapshot_download
import os

In [3]:

save_dir = '../results/baselines/'
repo_ids = [
    # encoder-decoder
    # t5
    't5-small',
    't5-base',
    't5-large',
    't5-3b',
    't5-11b',
#     # flan-t5
    'google/flan-t5-small',
    'google/flan-t5-base',
    'google/flan-t5-large',
    'google/flan-t5-xl'
    'google/flan-t5-xxl',
#     # decoder-only
#     # 124M, 355M, 774M, 1.5B
    'gpt2',
    'gpt2-medium',
    'gpt2-large',
    'gpt2-xl',
#     # open-source
    'huggyllama/llama-7b',
    'NousResearch/Llama-2-7b-hf'
    'mosaicml/mpt-7b',
    # pythia
    'EleutherAI/pythia-70m',
    'EleutherAI/pythia-160m',
    'EleutherAI/pythia-410m',
    'EleutherAI/pythia-1b',
    'EleutherAI/pythia-1.4b',
    'EleutherAI/pythia-2.8b',
    'EleutherAI/pythia-6.9b',
    'EleutherAI/pythia-12b',
    # instruction tuned pythia
    'databricks/dolly-v2-7b',
    # 
    'allenai/open-instruct-cot-7b',
    'allenai/open-instruct-flan-v2-7b',
    'allenai/open-instruct-dolly-7b',
    'allenai/open-instruct-oasst1-7b',
    'allenai/open-instruct-human-mix-7b',
    'allenai/tulu-7b',
    # sota chat instuct models
    'mistralai/Mistral-7B-v0.1',
    'mistralai/Mistral-7B-Instruct-v0.1',
    'HuggingFaceH4/mistral-7b-sft-alpha',  # sft(ultrachat1.5m)
    'HuggingFaceH4/mistral-7b-sft-beta', # sft(ultrachat200k)
    'HuggingFaceH4/zephyr-7b-alpha', # sft(ultrachat1.5m)+dpo
    'HuggingFaceH4/zephyr-7b-beta',  # sft(ultrachat200k)+dpo
    # embedding models
    'sentence-transformers/all-mpnet-base-v2',
    'BAAI/bge-large-en-v1.5',
    'jinaai/jina-embeddings-v2-base-en',
]

repo_ids = [
#     'codellama/CodeLlama-7b-hf',
#     'deepseek-ai/deepseek-coder-6.7b-base',
    'codellama/CodeLlama-7b-Python-hf',
    'codellama/CodeLlama-7b-Instruct-hf',
]

for repo_id in repo_ids:
    snapshot_download(repo_id=repo_id, local_dir=os.path.join(save_dir, repo_id))


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

In [2]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="SirNeural/flan_v2", 
    repo_type="dataset",
    local_dir="../data/raw_train/flan2022",
    local_dir_use_symlinks=False)

beaker_configs	env.yml  model_licenses  requirements.txt
data		eval	 open_instruct	 results
Dockerfile	images	 quantize	 scripts
ds_configs	LICENSE  README.md	 weight-diff-requirements.txt


In [12]:
from huggingface_hub import snapshot_download

repo_id = 'HuggingFaceH4/ultrachat_200k'
local_dir = '../data/raw_train/ultrachat_200k'

snapshot_download(
    repo_id=repo_id, 
    repo_type="dataset",
    local_dir=local_dir,
    local_dir_use_symlinks=False)



Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

'/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/data/raw_train/ultrachat_200k'

In [5]:
from transformers import LlamaTokenizer, LlamaTokenizerFast, AutoTokenizer, AddedToken


for model_name_or_path in [
    'results/baselines/huggyllama/llama-7b',
    'results/baselines/NousResearch/Llama-2-7b-hf',
    'results/baselines/mistralai/Mistral-7B-v0.1',
    'results/baselines/HuggingFaceH4/mistral-7b-sft-alpha',
    'results/baselines/HuggingFaceH4/mistral-7b-sft-beta',
    'results/baselines/HuggingFaceH4/zephyr-7b-alpha',
    'results/baselines/HuggingFaceH4/zephyr-7b-beta',
]:

    tokenizer_kwargs = {"use_fast": True,}
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, **tokenizer_kwargs)
    
    num_added_tokens = tokenizer.add_special_tokens({
        "bos_token": AddedToken("<s>", normalized=False, special=True),
        "eos_token": AddedToken("</s>", normalized=False, special=True),
        "unk_token": AddedToken("<unk>", normalized=False, special=True),
        "pad_token": AddedToken("<pad>", normalized=False, special=True),
    })

    tmp_tok_path = os.path.join(
        os.path.dirname(model_name_or_path),
        os.path.basename(model_name_or_path)+'_fixtok')
    
    tokenizer.save_pretrained(tmp_tok_path)
    tokenizer = AutoTokenizer.from_pretrained(tmp_tok_path, **tokenizer_kwargs)

    for s, s_tokenized in [
        ("Hi<s>Hey</s>sir<unk>what<pad><pad>", 
        ['▁Hi', '<s>', '▁Hey', '</s>', '▁sir', '<unk>', '▁what', '<pad>', '<pad>']),
    ]:
        assert(tokenizer.tokenize(s, add_special_tokens=False)==s_tokenized)

    
tokenizer




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

LlamaTokenizerFast(name_or_path='results/baselines/HuggingFaceH4/zephyr-7b-beta_fixtok', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='left', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<unk>', '<s>', '</s>', '<pad>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
num_added_tokens

0

In [10]:
from datasets import load_dataset


ds = load_dataset(
    'HuggingFaceH4/ultrachat_200k',
    cache_dir='../data/raw_train/ultrachat',
    split='train_sft')
ds

Found cached dataset parquet (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/../data/raw_train/ultrachat_200k/HuggingFaceH4___parquet/HuggingFaceH4--ultrachat_200k-75c6e299e27d1db5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


Dataset({
    features: ['prompt', 'prompt_id', 'messages'],
    num_rows: 207865
})